#1. Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!ls -al '/gdrive/My Drive/data'

#2. Korean Character

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
plt.rc('font', family='NanumBarunGothic')

#3. Load Data

In [ ]:
import pandas as pd

In [ ]:
file = '/gdrive/My Drive/data/data_2020_1_12.csv'
data = pd.read_csv(file)

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
#101,서울 105,기흥 110,목천 115,대전 120,황간 125,남구미 130,동김천 135,경주 140,부산
#101,서울(406.94) 105,기흥(387.19) 110,목천(329.91) 115,대전(271.94) 120,황간(222.38) 125,남구미(167.25) 130,동김천(192.00) 135,경주(68.26) 140,부산(0)
# Distance from Seoul : 0, 19.75, 77.03, 135, 184.56, 214.94, 239.69, 338.68, 406.94
# Distance from Seoul : 0, 20, 77, 135, 185, 240, 215, 339, 407
data['도착영업소코드'] = data['도착영업소코드'].map({105: 20, 110: 77, 115: 135, 120: 185, 125: 240, 130: 215, 135: 339, 140: 407})

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.rename(columns={'도착영업소코드':'거리'}, inplace=True)

In [ ]:
data_destination = data[data['거리'].isin([407])]

In [ ]:
data_destination

In [ ]:
data_time = data_destination.groupby(['집계시','요일','거리'])['통행시간'].mean()

In [ ]:
data_time

In [ ]:
data_distance = data_time.unstack(level=-1)

In [ ]:
data_distance

In [ ]:
data_distance.dropna()

In [ ]:
data_out = data_distance.reset_index()

In [ ]:
data_out

In [ ]:
#Data frame to list
data_list = data_out.values.tolist()

In [ ]:
data_list[:5]

In [ ]:
# Train Dataset
x_train = [ r[:2] for r in data_list]
y_train = [ r[-1] for r in data_list]

#3 SHow Graph

In [ ]:
x_train[:5], y_train[:5]

In [ ]:
x1 = [r[0] for r in data_list]
x2 = [r[2] for r in data_list]
y = [r[-1] for r in data_list]

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')
ax.set_title('Multi Variable Regression')
ax.set_xlabel('집계시')
ax.set_ylabel('요일')
ax.set_zlabel('통행시간')
plot = ax.scatter3D(x1,x2,y,c='r')
plt.show()

#4. multi Variable using Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
learning_rate = 1e-4
learning_epochs = 5000

In [ ]:
# Stochastic gradient descent (SGD) Optimizer
sgd = tf.keras.optimizers.SGD(learning_rate=learning_rate)
# Mean Square Error (MSE) loss function
mse = tf.keras.losses.mean_squared_error

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(2,)))

model.compile(loss=mse, optimizer=sgd)

In [ ]:
model.summary()

In [ ]:
# Train the model
history = model.fit(x_train, y_train, epochs=learning_epochs)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(history.history['loss'])
plt.title('Cost Gradient Decent')
plt.ylabel('Total Cost')
plt.xlabel('Number of Training')
plt.show()

In [ ]:
print("%20s %20s" %('step', 'Cost')+'\n')
for step in range(learning_epochs):
    if step % 100 == 0:
        cost_val = history.history['loss'][step]
        print("%20i %20.5f" %(step, cost_val))

In [ ]:
Time = 14 #@param {type:"slider", min:0, max:23, step:1}
Day = 3 #@param {type:"slider", min:0, max:6, step:1}

In [ ]:
time_condition = data_out['집계시'] == Time
day_condition = data_out['요일'] == Day
data_out[time_condition & day_condition]

In [ ]:
input = [ Time, Day ]
time = model.predict([input])
ml_time = time[0][0]
week_days = ['월', '화', '수', '목', '금', '토', '일']
print("%10s %10s %50s" % ('집계시', '요일', '서울에서 부산까지 통행시간')+'\n')
print("%10s %10s %50s" % ( str(Time), week_days[Day], str(ml_time)))